In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pickle
import web3
from web3 import Web3, HTTPProvider
import json
import json_numpy
import ipfs_api as ip
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, roc_auc_score
import os

In [10]:
class NumpyArrayEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

In [17]:
class Server : 
    def __init__(self, user_ids,data_path,epoch):
        self.user_id = user_ids ### Each client use it is specific ip to extract it is data from file
        self.contract_address = contract_address   ### contract address generated from server
        self.trained_model = None  ### to save model
        # w3 = Web3(HTTPProvider('http://127.0.0.1:1111'))
        self.abi = abi
        self.data_path = data_path
        self.x_test_user  = 0 
        self.epoch = epoch
        ####  Creating tmp dir to save data
        if not os.path.exists("./tmp/") :
            os.mkdir("./tmp/")
    
    
    def load_data(self):
        x_ip = pd.read_csv(self.data_path, index_col=0)
        encoder = OneHotEncoder(sparse_output=False)
        encoder = encoder.fit(np.array(x_ip['Attack_type'].unique()).reshape(-1, 1))
        Y = x_ip['Attack_type']
        Y = encoder.transform(np.array(Y).reshape(-1, 1))
        x_ip.drop(columns=['ip.src_host', 'Attack_label', 'Attack_type'], inplace=True)
        int_data = x_ip.select_dtypes(include=int).astype(float).to_numpy()
        float_data = x_ip.select_dtypes(include=float).to_numpy()
        obj_data = x_ip.select_dtypes(include=bool).astype(float).to_numpy()
        x_ip = np.concatenate([float_data, int_data, obj_data], axis=1)
        self.x_train_user, self.x_test_user, self.y_train_user, self.y_test_user = train_test_split(
            x_ip, Y, test_size=.1, random_state=42)

    def preprocess(self):
        BATCH_SIZE = 20
        SHUFFLE_BUFFER = 100
        PREFETCH_BUFFER = 10
        self.load_data()
        self.x_train_user = tf.convert_to_tensor(self.x_train_user)
        data_size = len(self.x_train_user)
        # print(len(self.x_train_user))
        self.x_test_user = tf.convert_to_tensor(self.x_test_user)
        self.y_train_user = tf.convert_to_tensor(self.y_train_user)
        self.y_test_user = tf.convert_to_tensor(self.y_test_user)
        self.x_train_user = tf.reshape(self.x_train_user, [-1, 46])
        self.x_test_user = tf.reshape(self.x_test_user, [-1, 46])
        self.y_train_user = tf.reshape(self.y_train_user, [-1, 15])
        self.y_test_user = tf.reshape(self.y_test_user, [-1, 15])
        self.train_dataset = tf.data.Dataset.from_tensor_slices((self.x_train_user, self.y_train_user))
        self.test_dataset = tf.data.Dataset.from_tensor_slices((self.x_test_user, self.y_test_user))
        self.train_dataset = self.train_dataset.repeat(3).shuffle(SHUFFLE_BUFFER).batch(BATCH_SIZE).prefetch(
            PREFETCH_BUFFER)
        self.test_dataset = self.test_dataset.repeat(3).shuffle(SHUFFLE_BUFFER).batch(BATCH_SIZE).prefetch(
            PREFETCH_BUFFER)


    def Model_Arch(self):
        model = tf.keras.models.Sequential([
            tf.keras.layers.Input(shape=(46, 1)),
            tf.keras.layers.Conv1D(64, 2, activation='relu', name='conv1d_1'),
            tf.keras.layers.BatchNormalization(name='batch_normalization_1'),
            tf.keras.layers.MaxPooling1D(pool_size=2, name='max_pooling1d_1'),
            # tf.keras.layers.Conv1D(64, 2, activation='relu', name='conv1d_2'),
            # tf.keras.layers.BatchNormalization(name='batch_normalization_2'),
            # tf.keras.layers.MaxPooling1D(pool_size=2, name='max_pooling1d_2'),
            tf.keras.layers.Conv1D(128, 2, activation='relu', name='conv1d_3'),
            tf.keras.layers.BatchNormalization(name='batch_normalization_3'),
            tf.keras.layers.Flatten(name='flatten'),
            tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), name='dense_1'),
            # tf.keras.layers.Dropout(0.1, name='dropout'),
            # tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), name='dense_2'),
            tf.keras.layers.Dense(15, activation='softmax', name='dense_3')
        ])
        model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
            metrics=[
                tf.keras.metrics.Accuracy(),
                tf.keras.metrics.Precision(),
                tf.keras.metrics.Recall(),
                tf.keras.metrics.AUC()
            ]
        )
        self.trained_model_arch = model.to_json() 
        print("Global Model Arch Sent")
        self.trained_model = model 
        ##### send model weights for first time 
        model.save_weights('./model.weights.h5')
        cid = ip.publish('./model.weights.h5')
        print(f"Model weights cid {cid}" )
        ml_contract.functions.setGlobalModel(cid).transact({'from':w3.eth.accounts[0]})
        print("Global Model Weights Sent")

    def Add_clients(self) : 
        for i,data_index in enumerate(self.user_id):
            tx_hash = ml_contract.functions.addClient(data_index).transact({'from':w3.eth.accounts[0]})  # Use user_id[i] for address
            print(f'Client {data_index} Added Succefully')

    def scale_model_weights(self, weight, scalar):
        '''function for scaling a models weights'''
        weight_final = []
        steps = len(weight)
        for i in range(steps):
            weight_final.append(scalar * weight[i])
        return weight_final



    def sum_scaled_weights(self, scaled_weight_list):
        '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
        avg_grad = list()
        #get the average grad accross all client gradients
        for grad_list_tuple in zip(*scaled_weight_list):
            layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
            avg_grad.append(layer_mean)
        return avg_grad

    def validate(self):
        ####################################################
        ############ Calculate Model Accureacy #############
        print(" Validating Global Model")
        self.preprocess()
        cce = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

        predictions  = self.trained_model.predict(self.x_test_user)    
        # Convert y_test_user and predictions to class indices
        true_classes = tf.argmax(self.y_test_user, axis=1).numpy()  # Convert to numpy for scikit-learn
        predicted_classes = tf.argmax(predictions, axis=1).numpy()  # Convert to numpy for scikit-learn
        
        # Calculate precision and recall
        precision = precision_score(true_classes, predicted_classes, average='weighted')
        recall = recall_score(true_classes, predicted_classes, average='weighted')
        accuracy = accuracy_score(true_classes, predicted_classes)
        auc = roc_auc_score(self.y_test_user, predictions, average='weighted', multi_class='ovr')

        print(f'Precision: {precision}  , Recall: {recall},   AUC: {auc} , Acuracy :{accuracy}')

        print("Finished")
        return {'AUC':auc, "Precision":precision, "Recall":recall, "Accuracy":accuracy }

    def send_model_hash(self):
        
        ################################
        ##### save model weights ####### 
        self.trained_model.save_weights(f'./tmp/global_add{self.epoch}.weights.h5')
        print("wieghtd  ",f'./tmp/global_add{self.epoch}.weights.h5')
        ################################
        ##### send model hash ##########
        cid = ip.publish(f'./tmp/global_add{self.epoch}.weights.h5')
        print(cid)
        ml_contract.functions.setGlobalModel( cid).transact({'from':w3.eth.accounts[0]})
        #### Remove weights 
        os.remove(f'./tmp/global_add{self.epoch}.weights.h5')
    
    def Start_train(self):
        Total_dataset = 0 
        scaled_weights = []
        ###########################################
        ######### Clear Flag before start##########
        # for i in range(len(self.user_id)):
        #     ml_contract.functions.clearNewModelReady(i).transact({'from':w3.eth.accounts[0]})
        ml_contract.functions.setIsStartTraining(False).transact({'from':w3.eth.accounts[0]})
        ###########################################
        ######### Define model Arch ###############
        self.Model_Arch() 


        ###########################################
        #### Send model arch as Json############### 
        ml_contract.functions.sendModelArch(json.dumps(self.trained_model_arch)).transact({'from':w3.eth.accounts[0]})

        ###########################################
        #### Add Clients To System ################ 
        self.Add_clients()

        ml_contract.functions.setTrainLoop(True).transact({'from':w3.eth.accounts[0]})
        
        model_state = []

        # ml_contract.functions.setTrainLoop(True).transact({'from':w3.eth.accounts[0]})


        for i in range(self.epoch) :

            print("#################################################")
     
            ###########################################
            ######### Start Clients training ########## 
            ml_contract.functions.setIsStartTraining(True).transact({'from':w3.eth.accounts[0]})
            
            
            ###########################################
            #### Loop throw All Client to get there results
            # Initialize a set to keep track of ready clients
            ready_clients_set = set()
            ready_clients_set.clear()
            while len(ready_clients_set) < len(self.user_id):
                time.sleep(2)
                print(f'Waiting {len(self.user_id) - len(ready_clients_set)} Clients to be ready')
                
                # Loop through each client
                for i in range(len(self.user_id)):
                    # Check if the client is ready and has not been counted yet
                    if ml_contract.functions.clients(i).call()[2] and i not in ready_clients_set:
                        print(f"Client {self.user_id} with index {i} Finished Training")
                        ready_clients_set.add(i)
            ###### Exit loop when all clients finish there training 
            ###########################################
            # for i in range(len(self.user_id)):
                # ml_contract.functions.clearNewModelReady(i).transact({'from':w3.eth.accounts[0]})

            

            ######### Start Clients training ########## 
            ml_contract.functions.setIsStartTraining(False).transact({'from':w3.eth.accounts[0]})
    
            ###########################################
            #### Get data len to be weighted model##### 
                
            Total_dataset = ml_contract.functions.sumDataLen().call()
            print("Total Dataset Size ---> ",Total_dataset)
    
            ####################################################
            ###### downlaod models Weights from ipfs 
            ## Loop throw all clients and download it is weights 
            ####################################################
            for i in range(len(self.user_id)) : 
    
                #### Get cid
                cid = ml_contract.functions.getModelHash(i).call()
                # if os.path.exists(f'./tmp/{cid}'):
                #     os.remove(f'./tmp/{cid}')
                print(cid)
                # cid = 'QmYxh2Fet6FjA86PZQbfdoSG9xbiMYTmhTRrQi5iPuZ4R4'
                ip.download(cid,'./tmp/')
                if os.path.exists(f'./tmp/x{i}.weights.h5'):
                    os.remove(f'./tmp/x{i}.weights.h5')
                ### Load  file 
                os.rename(f'./tmp/{cid}', f'./tmp/x{i}.weights.h5')
                self.trained_model.load_weights(f'./tmp/x{i}.weights.h5')
                #remove file after loading it 
                # os.remove(f'./tmp/{cid}')
                os.remove(f'./tmp/x{i}.weights.h5')
                # ml_contract.functions.
                
                ####################################################
                ############### Calculate scale #################### 
                # print(type(self.trained_model.weights))
                scale = ml_contract.functions.getDataLen(i).call() / Total_dataset
                w = self.trained_model.get_weights()
                # print(len(w))
    
                ####################################################
                #### Scale each model weight and save it############ 
                new_weight = self.scale_model_weights(w,scale )
                scaled_weights.append(new_weight)
                ml_contract.functions.clearNewModelReady(i).transact({'from':w3.eth.accounts[0]})
                print("Wainting Clients to Finish Training")
            ####################################################    
            ############### Aggregate weights ##################
            average_weights = self.sum_scaled_weights(scaled_weights)
    
            ####################################################
            ############## update global model ################# 
            self.trained_model.set_weights(average_weights)
            self.send_model_hash()
            

            model_state.append(self.validate())     
        ml_contract.functions.setTrainLoop(False).transact({'from':w3.eth.accounts[0]})

            
        
        return model_state
        
    

            
        

In [18]:
### Connect Ganache endpoint
w3 = web3.Web3(web3.HTTPProvider('http://127.0.0.1:7545'))
##
user_id = ['192.168.0.101',  '0' ]

### Contract Address
contract_address = '0x1c54d379603F38986B9c6c2F8216076F4324e24d'
######
abi = '[{"constant":true,"inputs":[],"name":"loop","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"serverNewModel","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"isStartTraining","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"model_architecture","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"clientCount","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"","type":"address"}],"name":"addressToIndex","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"","type":"uint256"}],"name":"clients","outputs":[{"name":"client_index","type":"uint256"},{"name":"new_model_hash","type":"string"},{"name":"new_model_ready","type":"bool"},{"name":"data_index","type":"string"},{"name":"data_len","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_state","type":"bool"}],"name":"setTrainLoop","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"getTrainLoop","outputs":[{"name":"loop","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"model","type":"string"}],"name":"setGlobalModel","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"getGlobalModel","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_data_index","type":"string"}],"name":"addClient","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"clientIndex","type":"uint256"},{"name":"newDataLen","type":"uint256"}],"name":"setDataLen","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"clientIndex","type":"uint256"}],"name":"getDataLen","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"sumDataLen","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"clientIndex","type":"uint256"},{"name":"modelHash","type":"string"}],"name":"SetModelHash","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"clientIndex","type":"uint256"}],"name":"getModelHash","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"clientIndex","type":"uint256"},{"name":"newModel","type":"string"}],"name":"updateModel","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"jsonString","type":"string"}],"name":"sendModelArch","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getModelArch","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_start","type":"bool"}],"name":"setIsStartTraining","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getIsStartTraining","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"clientIndex","type":"uint256"}],"name":"setNewModelReady","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"clientIndex","type":"uint256"}],"name":"clearNewModelReady","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}]'

ml_contract = w3.eth.contract(abi=abi, address=contract_address)


In [19]:
### Data Path 
epoch = 10
data_path = './Cleaned_data.csv'

test = Server(user_id,data_path,epoch)

In [1]:



state = test.Start_train()


NameError: name 'test' is not defined

In [ ]:
df = pd.DataFrame(state)
df.plot()

In [ ]:
for i in range(len(user_id)):
    ml_contract.functions.clearNewModelReady(i).transact({'from':w3.eth.accounts[0]})



In [ ]:
for i in range(5):
    print(ml_contract.functions.clients(i).call())

In [ ]:
for i in range(len(user_id)) : 
    print(i)